In [0]:
%run ./init

In [0]:
from datetime import datetime
import os

# # Set your storage account SAS token and name
# sasToken = "placeholder" # Replace with your actual SAS token
# sa = "erdccalearning" # Replace with your actual storage account name
# container_name = "nndemo" # Replace with your actual container name

# # Set the base URL for the container and directories
# base_url = f"wasbs://{container_name}@{sa}.blob.core.windows.net/AcceleratorSAPFiles/"
# spark.conf.set(f"fs.azure.account.auth.type.{sa}.dfs.core.windows.net", "SAS")
# spark.conf.set(f"fs.azure.sas.token.provider.type.{sa}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
# spark.conf.set(f"fs.azure.sas.nndemo.{sa}.blob.core.windows.net", f"{sasToken}")

sa = "stlpdel01dev"
container_name = "codeconverter" # Replace with your actual container name
spark.conf.set("fs.azure.account.auth.type", "CustomAccessToken")
spark.conf.set("fs.azure.account.custom.token.provider.class", spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName"))

base_path = f"abfss://{container_name}@{sa}.dfs.core.windows.net/AcceleratorSAPFiles/"

cleaned_directory_path = f"{base_url}CleanedSQLFiles/"
archive_exec_directory_path = f"{base_url}ArchivedExecutedSQLFiles/"

# Create an archive folder with the current date
current_date = datetime.now().strftime('%Y-%m-%d')
archive_exec_date_folder = f"{archive_exec_directory_path.rstrip('/')}/{current_date}/"
dbutils.fs.mkdirs(archive_exec_date_folder)

# List files in the cleansed directory
files = dbutils.fs.ls(cleaned_directory_path)

# Process each file in the cleaned directory
for file_info in files:
    file_path = file_info.path
    if file_info.isFile():
        print(f"Reading file: {file_path}")
        try:
            # Read file content
            file_content = read_file_content(file_path)
            if file_content is None:
                continue
            
            # Convert file content to a single string
            sql_code = "\n".join(file_content)
            
            # Split the SQL code into individual statements
            statements = sql_code.split(";")
            
            for statement in statements:
                statement = statement.strip()
                if statement:
                    # Execute each SQL statement individually
                    print(f"Executing SQL statement: {statement}")
                    spark.sql(statement)
            
            print(f"Successfully executed SQL code for file: {file_path}")
            
            # Archive the executed file to the archive directory
            archive_file_path = f"{archive_exec_date_folder}{os.path.basename(file_path)}"
            dbutils.fs.mv(file_path, archive_file_path)
            print(f"Successfully archived the executed file to: {archive_file_path}")
        except Exception as e:
            print(f"Error processing file {file_path}")
            print(f"Exception: {e}")
    else:
        print(f"Skipping directory: {file_path}")
